!pip install numpy>=1.23
!pip install pandas==1.3.5
!pip install scikit-learn==0.24.2
!pip install matplotlib==3.4.3
!pip install numba==0.55.1
!pip install scipy==1.7.3
!pip install gradio
!pip install urllib3==1.26.5
!pip install streamlit==1.2.0
!pip install protobuf==3.20.0
!pip install daal4py==2021.5.0
!pip install pyqt5==5.12.3 pyqtwebengine==5.12.1
!pip install spyder==5.1.5
!pip install --upgrade ruamel-yaml pathlib anaconda-project conda-repo-cli pyldavis numpy
!python -m pip install -U setuptools pip
!pip install imblearn

!pip install xgboost

## 確定要判定成 1 的規則先過濾出來，剩下的再用 AI 預測
## 討論 FN & FT 的狀態，是哪些情況抓不出來
## 最後再用 GenAI 來產生預警摘要

In [145]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.svm import SVC
from sklearn.metrics import classification_report

In [146]:
import pandas as pd

# Define file paths
auth_log_path = './data/authentication_log.csv'
operation_log_path = './data/operation_log.csv'
org_role_path = './data/org_role.csv'
system_role_path = './data/system_role.csv'
organization_path = './data/orgnazation.csv'

# Load the CSV files into pandas DataFrames
auth_log_df = pd.read_csv(auth_log_path)
operation_log_df = pd.read_csv(operation_log_path)
org_role_df = pd.read_csv(org_role_path)
system_role_df = pd.read_csv(system_role_path)
organization_df = pd.read_csv(organization_path)

In [147]:
# Define file paths for the anomaly text files
auth_log_anomaly_path = './data/authentication_log_anomaly.txt'
operation_log_anomaly_path = './data/operation_log_anomaly.txt'

# Read the text files into pandas DataFrames
auth_log_anomaly_df = pd.read_csv(auth_log_anomaly_path)
operation_log_anomaly_df = pd.read_csv(operation_log_anomaly_path)

In [148]:
# Merge authentication_log.csv and authentication_log_anomaly.txt
auth_anomaly_ids = [int(num.strip("[] ")) for num in auth_log_anomaly_df.columns]
auth_log_df['is_anomaly'] = 0
auth_log_df.loc[auth_log_df['id'].isin(auth_anomaly_ids), 'is_anomaly'] = 1

In [149]:
# Merge operation_log.csv and operation_log_anomaly.txt
operation_anomaly_ids = [int(num.strip("[] ")) for num in operation_log_anomaly_df.columns]
operation_log_df['is_anomaly'] = 0
operation_log_df.loc[operation_log_df['id'].isin(operation_anomaly_ids), 'is_anomaly'] = 1

In [150]:
auth_log_df.columns

Index(['id', 'user_account', 'role_id', 'user_id', 'action', 'description',
       'timestamp', 'IP', 'is_anomaly'],
      dtype='object')

In [151]:
operation_log_df.columns

Index(['id', 'user_account', 'role_id', 'user_id', 'action', 'description',
       'timestamp', 'IP', 'is_anomaly'],
      dtype='object')

In [152]:
auth_log_df.head()

,id,user_account,role_id,user_id,action,description,timestamp,IP,is_anomaly
0,1,user4@example.com,"1,2,3",4.0,login successful,NaN,2023-08-17 16:00:55.071858,192.168.44.216,0
1,2,user12@example.com,1,12.0,logout successful,NaN,2023-09-25 15:22:55.071880,192.168.101.214,0
2,3,user76@example.com,NaN,NaN,login failed,wrong account,2024-03-09 11:44:55.071892,192.168.172.52,0
3,4,user13@example.com,"2,3",13.0,login failed,wrong password,2023-12-06 22:50:55.071901,192.168.193.40,0
4,5,user81@example.com,"1,2,3",81.0,logout successful,NaN,2024-06-24 14:15:55.071910,192.168.116.148,0


In [153]:
import pandas as pd
import numpy as np

# 假日列表區
holidays = pd.to_datetime(['2024-01-01', '2024-02-28', '2024-04-04', '2024-05-01', '2024-06-25', '2024-09-28', '2024-10-10'])

def identify_day_type(date):
    if date in holidays:
        return 'holiday'
    elif date.weekday() >= 5:  # 5 表示星期六，6 表示星期天
        return 'weekend'
    else:
        return 'weekday'

In [154]:
# 特徵工程
# 1. 將類別型特徵轉換為數值
auth_log_df_ex = pd.DataFrame()
le = LabelEncoder()

auth_log_df_ex['user_action'] = auth_log_df['user_account'].astype(str) + '_' + auth_log_df['action'].astype(str)
auth_log_df_ex['user_ip'] = auth_log_df['user_account'].astype(str) + '_' + auth_log_df['IP'].astype(str)
# 對新的組合進行特徵編碼
auth_log_df_ex['user_action'] = le.fit_transform(auth_log_df_ex['user_action'])
auth_log_df_ex['user_ip'] = le.fit_transform(auth_log_df_ex['user_ip'])

auth_log_df_ex['user_account'] = le.fit_transform(auth_log_df['user_account'])
auth_log_df_ex['role_id'] = le.fit_transform(auth_log_df['role_id'])
auth_log_df_ex['user_id'] = le.fit_transform(auth_log_df['user_id'])
auth_log_df_ex['action'] = le.fit_transform(auth_log_df['action'])
auth_log_df_ex['description'] = le.fit_transform(auth_log_df['description'])
auth_log_df_ex['IP'] = le.fit_transform(auth_log_df['IP'])

In [155]:
# 2. 從 timestamp 中提取特徵 The day of the week with Monday=0, Sunday=6.
auth_log_df_ex['hour'] = pd.to_datetime(auth_log_df['timestamp']).dt.hour
auth_log_df_ex['minute'] = pd.to_datetime(auth_log_df['timestamp']).dt.minute
auth_log_df_ex['second'] = pd.to_datetime(auth_log_df['timestamp']).dt.second
auth_log_df_ex['dayofweek'] = pd.to_datetime(auth_log_df['timestamp']).dt.dayofweek
auth_log_df_ex['day_type'] = pd.to_datetime(auth_log_df['timestamp']).map(identify_day_type)
auth_log_df_ex['day_type'] = le.fit_transform(auth_log_df_ex['day_type'])

In [156]:
def detect_outliers(df, features):
    outlier_indices = []
    
    for c in features:
        # 1st quartile (25%)
        Q1 = np.percentile(df[c], 25)
        # 3rd quartile (75%)
        Q3 = np.percentile(df[c], 75)
        # IQR
        IQR = Q3 - Q1
        
        # outlier step
        outlier_step = 1.5 * IQR
        
        # Determine a list of indices of outliers for feature c
        outlier_list_col = df[(df[c] < Q1 - outlier_step) | (df[c] > Q3 + outlier_step)].index
        
        # append the found outlier indices for col to the list of outlier indices
        outlier_indices.extend(outlier_list_col)
        
    # select observations containing more than 2 outliers
    outlier_indices = list(set([x for x in outlier_indices if outlier_indices.count(x) > 2]))
    return outlier_indices

# 檢查 'hour', 'minute' 和 'second' 的異常值
outliers_to_remove = detect_outliers(auth_log_df_ex, ['hour', 'minute', 'second'])
auth_log_df_ex = auth_log_df_ex.drop(outliers_to_remove, axis=0)

In [157]:
# 3. 填補缺失值 (若有)
auth_log_df_ex = auth_log_df_ex.fillna(0)  # 或其他適當的填補方式

# 4. 取出 X and Y
X = auth_log_df_ex
Y = auth_log_df['is_anomaly'].values

In [158]:
X.head()

,user_action,user_ip,user_account,role_id,user_id,action,description,IP,hour,minute,second,dayofweek,day_type
0,150,44124,53,2,3,1,2,38720,16,0,55,3,0
1,28,3057,12,0,11,2,2,899,15,22,55,0,0
2,236,72248,82,7,109,0,0,16419,11,44,55,5,1
3,29,4416,13,5,12,0,1,20961,22,50,55,2,0
4,256,77851,88,2,80,2,2,3986,14,15,55,0,0


In [159]:
# 選擇特徵和目標
features = X[['user_action', 'user_ip', 'user_account', 'IP', 'user_id', 'role_id', 'action', 'hour', 'minute', 'second', 'dayofweek', 'day_type']]
target = Y

In [160]:
features.columns.values

array(['user_action', 'user_ip', 'user_account', 'IP', 'user_id',
       'role_id', 'action', 'hour', 'minute', 'second', 'dayofweek',
       'day_type'], dtype=object)

In [161]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# 分類特徵的處理
numeric_features = ['hour', 'minute', 'second']
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_features = ['user_action', 'user_ip', 'user_account', 'IP', 'user_id', 'role_id', 'action', 'dayofweek', 'day_type']
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# 建立處理管道
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [162]:
# 分割訓練集和測試集
X_train, X_test, Y_train, Y_test = train_test_split(features, target, test_size=0.3, random_state=42)

In [163]:
X_train.head()

,user_action,user_ip,user_account,IP,user_id,role_id,action,hour,minute,second,dayofweek,day_type
99377,0,462,0,24136,99,1,0,0,44,55,3,0
38991,292,89993,100,9802,91,3,2,17,2,55,4,0
27322,55,12257,21,15169,19,2,2,3,32,55,4,0
85141,83,21834,31,6942,109,7,0,1,32,55,4,0
65413,106,29566,38,46202,35,0,2,6,19,55,5,1


In [164]:
from sklearn.pipeline import make_pipeline
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier())
])

param_grid = {
    'classifier__max_depth': [3, 6, 10],
    'classifier__min_child_weight': [1, 5, 10],
    'classifier__gamma': [0.5, 1, 1.5, 2],
    'classifier__subsample': [0.6, 0.8, 1.0],
    'classifier__colsample_bytree': [0.6, 0.8, 1.0],
    'classifier__learning_rate': [0.01, 0.1, 0.2],
    'classifier__n_estimators': [100, 200, 300],
    'classifier__scale_pos_weight': [sum(Y_train == 0) / sum(Y_train == 1)]
}

optimal_params = {
    'max_depth': 14,
    'min_child_weight': 10,
    'gamma': 1,
    'subsample': 0.6,
    'colsample_bytree': 0.6,
    'learning_rate': 0.2,
    'n_estimators': 300,
    'scale_pos_weight': 21.580645161290324,
    'objective': 'binary:logistic'  # Assuming a binary classification problem
}

clf = XGBClassifier(**optimal_params, n_jobs=-1)

grid_search = GridSearchCV(model, param_grid, scoring='roc_auc', n_jobs=-1, cv=3, verbose=3)
grid_search.fit(X_train, Y_train)

In [165]:
clf.fit(X_train, Y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.6, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=1, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.2, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=14, max_leaves=None,
              min_child_weight=10, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=300, n_jobs=-1,
              num_parallel_tree=None, random_state=None, ...)

In [166]:
Y_pred = clf.predict(X_test)

In [167]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import plotly.figure_factory as ff
from sklearn.metrics import classification_report

In [168]:
# 計算正確率
accuracy = accuracy_score(Y_test, Y_pred)
#print(f"SVC 的正確率: {accuracy*100:.2f}%")
print(f"RandomForest 的正確率: {accuracy*100:.2f}%")

RandomForest 的正確率: 98.64%


In [169]:
print(classification_report(Y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99     28601
           1       0.84      0.87      0.86      1400

    accuracy                           0.99     30001
   macro avg       0.92      0.93      0.92     30001
weighted avg       0.99      0.99      0.99     30001



In [170]:
cm = confusion_matrix(Y_test, Y_pred)
cm_labels = [0, 1]

# Create the confusion matrix heatmap
fig = ff.create_annotated_heatmap(z=cm, x=cm_labels, y=cm_labels, colorscale='Blues')

# Update layout for better readability
fig.update_layout(
    title='Confusion Matrix',
    xaxis=dict(title='Predicted Label'),
    yaxis=dict(title='True Label')
)

fig.show()

In [179]:
probabilities = clf.predict_proba(X_test)

In [184]:
import numpy as np
indices_FN = np.where((Y_pred == 0) & (Y_test == 1))[0]
indices_TN = np.where((Y_pred == 1) & (Y_test == 1))[0]

In [188]:
# Extract the corresponding samples
y_true_selected_FN = X_test.iloc[indices_FN]
y_true_selected_TN = X_test.iloc[indices_TN]

In [189]:
FN = auth_log_df.loc[y_true_selected_FN.index]
TN = auth_log_df.loc[y_true_selected_TN.index]

In [190]:
# 把誤判為 0，但真實為 1 的成績秀出來
FN['score'] = probabilities[indices_FN, 1] * 100
TN['score'] = probabilities[indices_TN, 1] * 100

In [191]:
FN

,id,user_account,role_id,user_id,action,description,timestamp,IP,is_anomaly,score
57369,57370,user82@example.com,3,82.0,login failed,wrong account,2024-04-19 09:27:55.500125,192.168.79.23,1,4.389978
45529,45530,user89@example.com,1,89.0,login failed,wrong account,2024-06-13 02:42:55.412080,192.168.237.5,1,15.678914
21096,21097,user95@example.com,1,95.0,login successful,NaN,2023-10-09 03:39:55.230309,192.168.225.181,1,29.839304
13564,13565,user56@example.com,"1,2,3",56.0,login failed,wrong account,2024-04-20 00:01:55.174532,192.168.135.146,1,21.987402
82253,82254,user93@example.com,3,93.0,login successful,NaN,2023-10-10 18:57:55.686955,192.168.204.127,1,45.978455
...,...,...,...,...,...,...,...,...,...,...
71915,71916,user54@example.com,2,54.0,login successful,NaN,2024-04-19 06:46:55.608186,192.168.165.112,1,3.597020
40137,40138,user50@example.com,3,50.0,login successful,NaN,2023-08-31 16:42:55.372077,192.168.194.241,1,22.628965
35023,35024,user80@example.com,"2,3",80.0,login failed,wrong account,2023-08-31 05:21:55.334082,192.168.115.18,1,16.663715
32939,32940,user5@example.com,1,5.0,login successful,NaN,2023-08-04 02:22:55.318680,192.168.213.231,1,13.574487


In [192]:
TN

,id,user_account,role_id,user_id,action,description,timestamp,IP,is_anomaly,score
51387,51388,user105@example.com,3,105.0,login failed,wrong account,2024-01-01 02:19:55.455589,192.168.145.67,1,99.252090
86495,86496,user19@example.com,2,19.0,login successful,NaN,2024-04-06 02:28:25.718693,192.168.84.78,1,99.976105
86874,86875,user51@example.com,3,51.0,login failed,wrong account,2024-04-27 00:16:25.721718,192.168.143.100,1,99.997208
62756,62757,user84@example.com,1,84.0,login failed,wrong account,2023-11-19 09:11:55.540415,192.168.160.83,1,53.989834
74837,74838,user64@example.com,3,64.0,login successful,NaN,2023-08-09 00:40:55.630386,192.168.157.219,1,62.944542
...,...,...,...,...,...,...,...,...,...,...
14527,14528,user2@example.com,"1,2,3",2.0,login failed,wrong password,2024-07-01 05:51:25.181659,192.168.8.242,1,99.993774
28513,28514,user82@example.com,"2,3",82.0,login successful,NaN,2023-11-17 05:09:55.285865,192.168.24.11,1,93.632149
21371,21372,user30@example.com,3,30.0,login failed,wrong account,2023-12-01 23:23:25.232312,192.168.208.54,1,99.675568
45012,45013,user42@example.com,"1,2",42.0,login successful,NaN,2024-05-28 14:14:55.408270,192.168.153.87,1,54.004921


In [106]:
import plotly.express as px

# 分析密碼錯誤的登入失敗
password_errors = FN[FN['description'] == 'wrong password']

# 查看不同IP地址的登入嘗試次數
ip_attempts = FN['IP'].value_counts().reset_index()
ip_attempts.columns = ['IP', 'Counts']

# 查看不同角色的登入嘗試次數
role_attempts = FN['role_id'].value_counts().reset_index()
role_attempts.columns = ['Role_ID', 'Counts']

# 登入嘗試的時間分佈
FN['Hour'] = pd.to_datetime(FN['timestamp']).dt.hour
time_analysis = FN['Hour'].value_counts().sort_index().reset_index()
time_analysis.columns = ['Hour', 'Counts']

In [108]:
# 使用Plotly畫圖
fig1 = px.bar(ip_attempts, x='IP', y='Counts', title='Login Attempts by IP Address')
fig2 = px.bar(role_attempts, x='Role_ID', y='Counts', title='Login Attempts by User Role')
fig3 = px.bar(time_analysis, x='Hour', y='Counts', title='Login Attempts by Hour')

fig1.show()
fig2.show()
fig3.show()

In [109]:
print("密碼錯誤的登入失敗次數：", len(password_errors))
print("各IP地址的登入嘗試次數：\n", ip_attempts)
print("各角色的登入嘗試次數：\n", role_attempts)
print("登入嘗試的時間分佈：\n", time_analysis)

密碼錯誤的登入失敗次數： 2
各IP地址的登入嘗試次數：
                   IP  Counts
0     192.168.102.40       3
1    192.168.213.231       3
2     192.168.62.215       3
3      192.168.237.5       3
4    192.168.151.212       3
..               ...     ...
142    192.168.2.162       1
143  192.168.197.204       1
144   192.168.119.82       1
145   192.168.13.125       1
146    192.168.76.59       1

[147 rows x 2 columns]
各角色的登入嘗試次數：
   Role_ID  Counts
0       3      57
1       1      49
2       2      34
3   1,2,3      19
4     1,3      10
5     2,3       8
6     1,2       7
登入嘗試的時間分佈：
     Hour  Counts
0      0       5
1      1      10
2      2      12
3      3       7
4      4       5
5      5      11
6      6      14
7      7       8
8      8       2
9      9      12
10    10       9
11    11       6
12    12       1
13    13       3
14    14       8
15    15      19
16    16       6
17    17       5
18    18       9
19    19       6
20    20       4
21    21      14
22    22       2
23    23       6
